<a href="https://colab.research.google.com/github/m-numan1/100-Deep-Learning-Projects/blob/main/Sentiment_analysis_on_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Sentiment Analysis
**Steps**
 * we will use IMDB dataset
 * preprocess dataset(tokenize,tolowercase,remove punctuation, build vocab,convert words to indices, padding to same length)
 * we will use pretrained  model,
 * we will add our linear part to model
 * training
 * evaluation

In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from collections import Counter
import pandas as pd

In [2]:
# importing libraries for word tokenizer and preprocessing
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import re


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# downloading dataset direclty from kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#!/bin/bash
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.41GB/s]


In [6]:

!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [7]:
df = pd.read_csv('/content/IMDB Dataset.csv')

In [8]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
df.sentiment = df.sentiment.replace({'positive':1,'negative':0})

/tmp/ipython-input-9-1948547579.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.sentiment = df.sentiment.replace({'positive':1,'negative':0})


In [10]:
x = df.review
y = df.sentiment

In [11]:
# making document of df column sentences
sentences = []
for sentence in x:
  sentences.append(sentence)

In [13]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

**STEPS**

* tokenize(making of single words)
* create vocabulary of those words
* now make embeddings

In [14]:
tokens = [word_tokenize(sentence) for sentence in sentences]

In [15]:
tokens[:1]

[['One',
  'of',
  'the',
  'other',
  'reviewers',
  'has',
  'mentioned',
  'that',
  'after',
  'watching',
  'just',
  '1',
  'Oz',
  'episode',
  'you',
  "'ll",
  'be',
  'hooked',
  '.',
  'They',
  'are',
  'right',
  ',',
  'as',
  'this',
  'is',
  'exactly',
  'what',
  'happened',
  'with',
  'me.',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  'The',
  'first',
  'thing',
  'that',
  'struck',
  'me',
  'about',
  'Oz',
  'was',
  'its',
  'brutality',
  'and',
  'unflinching',
  'scenes',
  'of',
  'violence',
  ',',
  'which',
  'set',
  'in',
  'right',
  'from',
  'the',
  'word',
  'GO',
  '.',
  'Trust',
  'me',
  ',',
  'this',
  'is',
  'not',
  'a',
  'show',
  'for',
  'the',
  'faint',
  'hearted',
  'or',
  'timid',
  '.',
  'This',
  'show',
  'pulls',
  'no',
  'punches',
  'with',
  'regards',
  'to',
  'drugs',
  ',',
  'sex',
  'or',
  'violence',
  '.',
  'Its',
  'is',
  'hardcore',
  ',',
  'in',
  'the',
  'classic',
  'use',
  'of',
  't

In [16]:
def clean_tokens(tokenized_sentences):
    cleaned = []
    for sentence in tokenized_sentences:
        cleaned_sentence = [
            token.lower()
            for token in sentence
            if token.isalpha()  # keeps only alphabetic words (no punctuation/numbers)
        ]
        cleaned.append(cleaned_sentence)
    return cleaned

tokens = clean_tokens(tokens)  # overwrite the original with cleaned version


In [17]:
tokens[:1]

[['one',
  'of',
  'the',
  'other',
  'reviewers',
  'has',
  'mentioned',
  'that',
  'after',
  'watching',
  'just',
  'oz',
  'episode',
  'you',
  'be',
  'hooked',
  'they',
  'are',
  'right',
  'as',
  'this',
  'is',
  'exactly',
  'what',
  'happened',
  'with',
  'br',
  'br',
  'the',
  'first',
  'thing',
  'that',
  'struck',
  'me',
  'about',
  'oz',
  'was',
  'its',
  'brutality',
  'and',
  'unflinching',
  'scenes',
  'of',
  'violence',
  'which',
  'set',
  'in',
  'right',
  'from',
  'the',
  'word',
  'go',
  'trust',
  'me',
  'this',
  'is',
  'not',
  'a',
  'show',
  'for',
  'the',
  'faint',
  'hearted',
  'or',
  'timid',
  'this',
  'show',
  'pulls',
  'no',
  'punches',
  'with',
  'regards',
  'to',
  'drugs',
  'sex',
  'or',
  'violence',
  'its',
  'is',
  'hardcore',
  'in',
  'the',
  'classic',
  'use',
  'of',
  'the',
  'br',
  'br',
  'it',
  'is',
  'called',
  'oz',
  'as',
  'that',
  'is',
  'the',
  'nickname',
  'given',
  'to',
  'th

In [18]:
# i have a list of tokens lists
# Flatten the list
flat_tokens = [token for sublist in tokens for token in sublist]

In [19]:
flat_tokens[:10]

['one',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching']

In [20]:

vocab = {'<pad>':0,'<unkn>':1 }
for word in Counter(flat_tokens).keys():
  if word not in vocab:
    vocab[word] = len(vocab)

len(vocab)

97035

**NOTE**

* This above steps we were doing to make a vocabulary
* now we will make a sentences list tokens and convert those into numerical based on vocab

In [21]:
# i want to make a numerical list from tokens
numerical_sentences = []
def word_to_index(tokens, vocab):
    numerical_sentences = []
    for sentence in tokens:
        numerical_sentence = []
        for word in sentence:
            if word in vocab:
                numerical_sentence.append(vocab[word])
            else:
                numerical_sentence.append(vocab['<unkn>'])
        numerical_sentences.append(numerical_sentence)  # ← outside the inner loop
    return numerical_sentences

numerical_sentences = word_to_index(tokens, vocab)

In [70]:
# padding the numerical sentences

from torch.nn.utils.rnn import pad_sequence
# Convert to tensors
tensor_sentences = [torch.tensor(s) for s in numerical_sentences]

# Pad
padded = pad_sequence(tensor_sentences, batch_first=True, padding_value=0, )
padded = padded[:, :300]

In [56]:
padded.shape

torch.Size([50000, 100])

In [23]:
len(padded)

50000

In [57]:
y_labels = torch.tensor(y, dtype =  torch.long)

In [58]:
x_train, x_test, y_train, y_test = train_test_split(padded, y_labels, test_size=0.2, random_state=42)

In [59]:
train_data = TensorDataset(x_train, y_train)
test_data = TensorDataset(x_test, y_test)

In [60]:
train_data_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=32, shuffle=True)

**Next Step**

* Data Preprocessing done now we will make the model

In [71]:
# Designing model

class SentimentClassifier(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,padding_idx):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim,padding_idx= padding_idx )
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

    self.fc2 = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    embedded = self.embedding(x)
    lstm_out,(hidden, _) = self.lstm(embedded)
    output = F.dropout(self.fc2(hidden[-1]), 0.3)
    # output = self.fc2(hidden.squeeze(0))

    return output


In [62]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [75]:

model = SentimentClassifier(len(vocab), 100, 512, 2, vocab['<pad>']).to(device)

epoch = 15
leaarning_rate = 0.001

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=leaarning_rate)

In [76]:
# training loop

for epoch in range(epoch):
  total_loss = 0
  for review , label in train_data_loader:
    review, label = review.to(device), label.to(device)
    optimizer.zero_grad()
    output = model(review)

    loss = loss_function(output, label)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"epoch: {epoch+1} loss: {total_loss/len(train_data_loader)}")

epoch: 1 loss: 0.6998803044319153
epoch: 2 loss: 0.6160419598817826
epoch: 3 loss: 0.4316791366815567
epoch: 4 loss: 0.34296802608966825
epoch: 5 loss: 0.27968388097286223
epoch: 6 loss: 0.22327392486184836
epoch: 7 loss: 0.1671778010159731
epoch: 8 loss: 0.12871530753485858
epoch: 9 loss: 0.10318958973810077
epoch: 10 loss: 0.08905411238113883
epoch: 11 loss: 0.0793774754159851
epoch: 12 loss: 0.07971142908786423
epoch: 13 loss: 0.08016838446902694
epoch: 14 loss: 0.0726763469747384
epoch: 15 loss: 0.06991481758407317


In [77]:
model.eval()

accuracy = 0
total = 0
with torch.no_grad():

  for review, label in test_data_loader:
    review, label = review.to(device), label.to(device)
    output = model(review)
    _, predicted = torch.max(output.data, 1)
    total += label.size(0)
    accuracy += (predicted == label).sum().item()

  print(f"Accuracy: {100 * accuracy / total}%")

Accuracy: 79.02%


In [95]:


def preprocess_review(review, tokenizer, vocab, max_len=300):
    # Tokenize (split to words)
    tokens = tokenizer(review.lower())  # or use your own tokenizer logic

    # Convert to indices
    indices = [vocab.get(token, vocab['<unkn>']) for token in tokens]

    # Convert to tensor
    tensor = torch.tensor(indices)

    # Pad or truncate to max_len
    if len(tensor) > max_len:
        tensor = tensor[:max_len]
    else:
        padding = torch.full((max_len - len(tensor),), vocab['<pad>'])
        tensor = torch.cat([tensor, padding])

    return tensor.unsqueeze(0)  # Add batch dimension


In [96]:
def predict_sentiment(review, model, tokenizer, vocab, max_len=300):
    model.eval()
    with torch.no_grad():
        input_tensor = preprocess_review(review, tokenizer, vocab, max_len).to(device)
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1)
        predicted = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted].item()
        sentiment = 'Positive' if predicted == 1 else 'Negative'
        return sentiment, confidence


In [105]:
review_text = "The movie was good!"
sentiment, confidence = predict_sentiment(review_text, model, word_tokenize, vocab)
print(f"Sentiment: {sentiment} (Confidence: {confidence:.2f})")


Sentiment: Negative (Confidence: 0.61)
